# SenSwiss 2021

Code to run in parallel with the acquisition of images in SpinView and to control the pump.

Instructions: 

* Check that the **exposure time** of the camera is constant 11113us
* Check you have a folder named `/focus` at the main path
* Create a **new folder for the measurement** `/images/NEW_FOLDER_FOR_NEW_TEST` (this needs to be done each time, not in advance!!! (code checks last created folder))
* Follow the instructions below
* For acquisition in SpinView, save the images in the folder you created in step 3 as type .tiff
* If the chip moves at some point, take another image and re-run press_ROI
* Look that everything is okay in the prints
* Press ctrl-j (for like 1-2seconds) when you want to finish computing the preprocessing and analysis and save the results in result.csv, which will be placed in the folder of step 3.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import keyboard
import sys
import serial
import time
import numpy as np

import run_parallel
from pump import *
from run_parallel import describe_folders, press_ROIs, run_analysis
from processing.processing_functions import compute_concentration_linear, compute_concentration_exponential

## 0. Pump

To connect the pump to the computer (only done at the very beginning of times).

In [ ]:
# Open serial connection -> check COM port on your device
lsp = serial.Serial('COM3', 9600, timeout=1000)  # TODO: change the comport depending on your computer
print('LSPone connected on ', lsp.name)

# Initialise LSPone normally already done
initialise_LSPone(lsp)

In [ ]:
# 0. General Cleaning # this were i start
# 2: PBS, 3: Air, 4: Ethanol   (order of the tubes in the pump)
general_cleaning_procedure(lsp)

## Starting the measurements

#### 1. Take a photo with SpinView of the spots -> save at `/focus`

#### 2. Create a folder named `/images/NEW_FOLDER_FOR_NEW_TEST` and set everything up in SpinView: photo each 2000ms for 240000ms

#### 3. Aspire the Blocking Buffer

In [ ]:
# Sucking BB at 50 ul/min
pick_BB(lsp)

#### 4. Select ROI (in the meantime)


In [ ]:
# Select ROI
ORIGINAL_FOLDER, IMG_PROCESSED_FOLDER, IMG_FOLDER, DIR_ROI, DIR = describe_folders()
ROIs = press_ROIs(DIR_ROI, ORIGINAL_FOLDER)

#### 5. Flow the sample

#### Do not forget to change n to 4

In [ ]:
# Push and pull sample at 50 ul/min
push_pull_sample_n_times(lsp, n = 4)

#### 6. Start running preprocess and analysis

In [ ]:
# Run processing and analysis
run_analysis(ROIs, IMG_FOLDER=IMG_FOLDER, DIR=DIR, window_size=5, framerate=2, threshold=140)

In [ ]:
# TODO: add 15seconds timer

#### 7. SpinView start recording -> save at `/images/NEW_FOLDER_FOR_NEW_TEST` as soon as you see liquid

#### 8. View results (change name)

In [ ]:
# Loading results
result_df = pd.read_csv('./images/20210830_10ng_5/result.csv')   # TODO CHANGE NAME
result_df['Signal'].plot()

In [ ]:
time_step = 2*5
y = result_df['Signal']
x = np.arange(0, len(y) * time_step, time_step)

plt.plot(x, y)

conc_linear = compute_concentration_linear(x, y)
conc_exp = compute_concentration_exponential(x, y)

In [ ]:
# Running in parallel (if you want to run all at once)
# run_parallel.main()

#### 9. Clean pump

In [ ]:
# 0. General Cleaning 
# 2: PBS, 3: Air, 4: Ethanol   (order of the tubes in the pump)
general_cleaning_procedure_fast(lsp)

In [ ]:
empty_syringe(lsp)

In [ ]:
pick_BB_again(lsp)